<h1 style="text-align: center">
Deep Learning HW4 </br>
Poem generation,fine tuning
</h1>



  <style>
    table {
      width: 100%;
      border-collapse: collapse;
    }

    th, td {
      border: 1px solid #dddddd;
      text-align: left;
      padding: 8px;
    }

    th {
      background-color: #f2f2f2;
    }

    .box {
      border: 1px solid #000;
      padding: 10px;
      width: 400px; /* Adjust the width as needed */
      margin: 20px auto;
    }
  </style>
</head>
<body>

<div class="box">
  <table>
    <tr>
      <th colspan="2">Personal Info</th>
    </tr>
    <tr>
      <td>First Name:</td>
      <td>Ali</td>
    </tr>
    <tr>
      <td>Last Name:</td>
      <td>Nikkhah</td>
    </tr>
    <tr>
      <td>Student Number:</td>
      <td>99102445</td>
    </tr>
    <tr>
      <td>Git:</td>
      <td><a href="https://github.com/AliNikkhah2001/DataScience02" target="_blank">https://github.com/AliNikkhah2001/DeepLearning02</a></td>
    </tr>
  </table>
</div>



# Poem generator using Ferdowsi poems dataset

## Data Import and initialization

In [1]:
%%capture
!pip install keras_preprocessing
!pip install transformers


In [2]:
%%capture
import tensorflow as tf
import numpy as np
import math
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [3]:

from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config
from IPython import display
     

## Open text file and make dataframe 

In [4]:
!git lfs install
!git clone https://huggingface.co/HooshvareLab/gpt2-fa-poetry

Git LFS initialized.
fatal: destination path 'gpt2-fa-poetry' already exists and is not an empty directory.


In [5]:
data_path='/kaggle/input/ferdowsi/ferdousi.txt'

In [6]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
lines = lines[2:]


In [35]:
model_name_or_path = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    bos_token='<s>', 
    eos_token='</s>', 
    pad_token='<pad>',
    unk_token='<unk>',
    padding_side="left"
)
tokenizer.add_special_tokens({
    "bos_token": '</s>',
    "eos_token": '</s>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

config = AutoConfig.from_pretrained(
    model_name_or_path,
    bos_token_id=tokenizer("<s>")["input_ids"][0], 
    eos_token_id=tokenizer("</s>")["input_ids"][0], 
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)



In [36]:
tokenizer.save_pretrained("/kaggle/working/gpt2-fa/")
config.save_pretrained("/kaggle/working/gpt2-fa/")

In [8]:
df_input=[]
df_target=[]
df_concat=[]
for i in range(0, len(lines)-3, 2):
    input_text= lines[i] 
    target_text = ' <s> '+ lines[i+1]+'   ' +lines[i+2] + '    ' + lines[i+3]+'  </s>  '
    df_input.append(input_text)
    df_target.append(target_text)
    df_concat.append(input_text+target_text)

In [9]:
print(len(df_input))
print(len(df_target))

49607
49607


In [10]:
idx = 50
print(f'{df_input[idx]}')
print(f'{df_target[idx]}')
print(f'{df_concat[idx]}')

همی بر شد آتش فرود آمد آب
 <s> همی گشت گرد زمین آفتاب   گیا رست با چند گونه درخت    به زیر اندر آمد سرانشان ز بخت  </s>  
همی بر شد آتش فرود آمد آب <s> همی گشت گرد زمین آفتاب   گیا رست با چند گونه درخت    به زیر اندر آمد سرانشان ز بخت  </s>  


In [11]:

print(tokenizer.encode("<s>گیا رست با چند گونه درخت    به زیر اندر آمد سرانشان ز بخت</s>"))
print(tokenizer.encode("<s>"))
print(tokenizer.encode("</s>"))
print(tokenizer.encode("<pad>"))
print(tokenizer.encode("<|startoftext|>"))
print(tokenizer.encode("<sep>"))

[0, 423, 268, 2630, 314, 638, 1752, 3087, 21147, 303, 730, 15233, 2199, 3325, 549, 355, 11653, 2]
[0]
[2]
[1]
[6]
[9]


## MultiGPU setup

In [12]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat Jan 13 07:49:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
import torch

def set_up_two_gpus():
    available_gpus = torch.cuda.device_count()
    if available_gpus < 2:
        raise ValueError("Insufficient number of GPUs available. Need at least two GPUs.")

    device_1 = torch.device("cuda:0")
    device_2 = torch.device("cuda:1")

    print(f"Available GPUs: {available_gpus}")
    print(f"Setting GPU 0 as device: {device_1}")
    print(f"Setting GPU 1 as device: {device_2}")

    return device_1, device_2

# Check and print if the GPUs are working by performing a computational operation
def check_gpu(device):
    tensor = torch.randn(1000, 1000, device=device)
    result = torch.matmul(tensor, tensor)
    return result

# Usage
device_1, device_2 = set_up_two_gpus()
result_1 = check_gpu(device_1)
result_2 = check_gpu(device_2)
print("Results from GPU 0:", result_1)
print("Results from GPU 1:", result_2)


Available GPUs: 2
Setting GPU 0 as device: cuda:0
Setting GPU 1 as device: cuda:1
Results from GPU 0: tensor([[ 75.6320,  31.3526,  64.5673,  ...,  13.4411, -11.9168,  34.0021],
        [-18.1419,  -7.9399,  35.6506,  ..., -75.3446,  26.5143, -28.9744],
        [ -7.3059,  42.9290, -25.3374,  ...,  -9.7965,  27.7825, -22.4071],
        ...,
        [ 13.8088,   7.8570,  12.9168,  ...,  29.4321,  32.3362, -18.7860],
        [-15.4928,  24.0701, -32.4685,  ...,  17.1769, -24.8387, -59.2409],
        [ 37.2607,  21.3787, -12.6041,  ...,  34.6673,  -6.4960,  31.2525]],
       device='cuda:0')
Results from GPU 1: tensor([[ 1.4109e+01,  1.4025e+00, -3.8026e+01,  ...,  1.7126e+01,
         -7.0841e+00, -1.6453e+01],
        [ 3.0766e+01,  8.0499e+00,  1.6201e+01,  ..., -3.0754e+01,
         -4.8530e+01, -1.5427e+01],
        [-2.0524e+01,  2.5253e+01,  1.7683e+01,  ...,  5.1375e+01,
          2.5864e+01,  8.7076e+00],
        ...,
        [-3.0036e+00, -1.7768e+01, -2.9617e+00,  ..., -4.5509e

In [14]:
# Sample input text
input_text = df_input[50]
# Tokenize and encode the input text
input_encoding = tokenizer.encode(input_text)

# Decode the tokenized input back to text
decoded_input = tokenizer.decode(input_encoding)

# Print results
print("Original Text:", input_text)
print("Tokenized Input:", input_encoding)
print("Decoded Input:", decoded_input)


Original Text: همی بر شد آتش فرود آمد آب
Tokenized Input: [272, 398, 327, 403, 2466, 2638, 2199, 797]
Decoded Input: همی بر شد آتش فرود آمد آب


In [15]:
from torch.utils.data import Dataset  
import torch
torch.manual_seed(42)
class MTGDataset(Dataset):
    def __init__(self, input_texts, tokenizer, max_length=1024):
        self.tokenizer = tokenizer  
        self.input_ids = []
        self.attn_masks = []

        for txt in input_texts:
            encodings_dict = tokenizer(txt ,
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [16]:
from torch.utils.data import random_split
max_seq = 256
dataset = MTGDataset(df_concat, tokenizer, max_length=max_seq)
# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = (len(dataset) - train_size)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {len(train_dataset)} samples for training, and {len(val_dataset)} samples for validation testing'

'There are 44646 samples for training, and 4961 samples for validation testing'

In [17]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('/kaggle/working/gpt2-fa/', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("/kaggle/working/gpt2-fa/", config=configuration)
model.resize_token_embeddings(len(tokenizer))


# This step is optional but will enable reproducible runs.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [18]:
epochs = 3
warmup_steps = 1e2
sample_every = 300

In [19]:
from transformers import AdamW

# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters.
optimizer = AdamW(
    model.parameters(),
    lr=5e-4,
    eps=1e-8,
    no_deprecation_warning=True
)

In [20]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=8
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=8
)

In [21]:
from transformers import get_linear_schedule_with_warmup
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps)


In [22]:
from torch.nn import DataParallel
# Define a function to set up multi-GPU training
def multi_gpu_setup(model):
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs")
        model = DataParallel(model)
    return model

# Use the function to set up multi-GPU training
model = multi_gpu_setup(model)

Using 2 GPUs


In [23]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cuda':
    print(f"Memory Allocated: {torch.cuda.memory_allocated(device)/1024**3:.2f} GB")
    print(f"Memory Cached: {torch.cuda.memory_cached(device)/1024**3:.2f} GB")
else:
    print("CUDA is not available. Switching to CPU.")


Using device: cuda
Memory Allocated: 0.01 GB
Memory Cached: 0.02 GB


/opt/conda/lib/python3.10/site-packages/torch/cuda/memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [24]:

sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to('cpu')

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

جوان گرچه دانا بود با گهر
Example output: جوان گرچه دانا بود با گهربار و با فراستش این کار را کرد و مردم را برانداخت به تباهی. پس از او، نوبت به یزدگرد رسید. سپاه به سوی غرب رفت. سپاه عرب در آن سوی جیحون (که در زبان فارسی از
Example output: جوان گرچه دانا بود با گهربار عشق» اما در این میان که چه چیزی از قلم افتاده است. در این میان می‌بینم «ای مردم، چرا نباید در دل آتش به گریه بیفتد؟» تا این که یک نفر از مردم «هست
Example output: جوان گرچه دانا بود با گهربار که این هم یک بار خیلی هم آدم را خسته نکرد و از فرط خستگی به خودش آمد. او را با این حال صدا می‌کند، می‌گوید: «با همین حال می‌توانم به خودم امید
Example output: جوان گرچه دانا بود با گهربار در کوچه پس‌کوچه‌ها قدم زد. با صدای آواز دلکش و نغمه‌هایی دلنشین از بنان. به آواز روح می‌افروزم و آرام می‌نشیند. می‌آید و می‌نشیند
Example output: جوان گرچه دانا بود با گهربار و با خود می‌گوید من هنوز به اندازهٔ تو برای زندگی کردن احتیاج دارم (پس من هم به همین نسبت احتیاج دارم) و فقط وقتی بزرگ شدم این مشکل‌ها را برطرف و از آن‌ها


In [25]:
%%capture
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
torch.cuda.empty_cache()

In [27]:
from tqdm.notebook import tqdm


In [30]:

training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    learning_rate=5e-5,
    logging_steps=500,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=1000,
    # Add distributed training options
    per_device_eval_batch_size=32,
    fp16=True,  # Enable mixed-precision training if supported
    dataloader_num_workers=16,  # Adjust based on your system capabilities
    report_to="tensorboard",  # You can use TensorBoard for logging
)

# Initialize Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)




In [31]:
import random
import time
import datetime
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))
total_t0 = time.time()
training_stats = []
model = model.to(device)
for epoch_i in tqdm(range(training_args.num_train_epochs),position=0):
    print(f'Beginning epoch {epoch_i + 1} of {training_args.num_train_epochs}')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        model.zero_grad()
        outputs = model(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)
        loss = outputs.loss
        loss = loss.mean()
        batch_loss = loss
        total_train_loss += batch_loss
        # Get sample every 100 batches.
        if step % sample_every == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print()
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')
            model.eval()
            sample_input = df_input[np.random.randint(0, len(df_input))]
            print(sample_input)
            sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
            sample_input_ids = sample_input_ids.to(device)
            sample_outputs = model.module.generate(
                input_ids=sample_input_ids,
                do_sample=True,
                top_k=50,
                max_length=50,
                top_p=0.95,
                num_return_sequences=1
            )
            for i, sample_output in enumerate(sample_outputs):
                gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
                gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
                gen_sample_output = gen_sample_output.replace("<s>", "")
                gen_sample_output = gen_sample_output.replace("</s>", "")
                gen_sample_output = gen_sample_output.replace("<sep>", "\n")
                print(f'Example output: {gen_sample_output}')
            model.train()
        loss.backward()
        optimizer.step()
        scheduler.step()
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print()
    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')
    print()
    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_masks, labels=b_labels)
            loss = outputs.loss
            loss = loss.mean()
            batch_loss = loss
        total_eval_loss += batch_loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    print()
    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')
    print()
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')

  0%|          | 0/2 [00:00<?, ?it/s]

Beginning epoch 1 of 2


  0%|          | 0/5581 [00:00<?, ?it/s]


Batch 300 of 5581. Loss:0.5782329440116882. Time:0:01:59
جوان گرچه دانا بود با گهر
Example output: جوان گرچه دانا بود با گهر   که برتهٔ خویش کز شاه جهان بود   چو باد آمد بر سر او  

Batch 600 of 5581. Loss:0.5162903070449829. Time:0:04:02
ببستند یارانش یکسر کمر
Example output: ببستند یارانش یکسر کمر  بفرمود و بر هر جای   بدانگه که آمد لشکر چو باد    همه پاک یزدان بر باد  

Batch 900 of 5581. Loss:0.526287317276001. Time:0:06:04
نوندی ز گفتار کارآگهان
Example output: نوندی ز گفتار کارآگهان  نگردد نه هنگام مهتران کهن   وزان سان که بشنید زان گونه جنگ    همی گیرد به یک سو بگفت و چون شنید  

Batch 1200 of 5581. Loss:0.49642834067344666. Time:0:08:07
سرانجام اگر راه جویی بداد
Example output: سرانجام اگر راه جویی بداد  که چون باده در جنگ جویی بداد   یکی تیغ زن را بر کرد و برکشید    پس پشت او از دشت در گذشت  

Batch 1500 of 5581. Loss:0.4387779235839844. Time:0:10:09
بسی آفرین کرد بر شهریار
Example output: بسی آفرین کرد بر شهریار  به ابر اندر آمد و باد بهار   ز ایران بسی نامدارانش    ز هر نام

  0%|          | 0/621 [00:00<?, ?it/s]


Validation loss: 0.3681398332118988. Validation Time: 0:01:37

Beginning epoch 2 of 2


  0%|          | 0/5581 [00:00<?, ?it/s]


Batch 300 of 5581. Loss:0.3541761338710785. Time:0:02:02
به تاراج داد آن همه خواسته
Example output: به تاراج داد آن همه خواسته  زمین گشت آراسته چون بهشتی شد آراسته   چو این نامه برخواند پیشش دبیر    به دستوری تیز و پاسخ دلپذیر  

Batch 600 of 5581. Loss:0.3396536111831665. Time:0:04:05
سپاهش بکوه کنابد شود
Example output: سپاهش بکوه کنابد شود  وزو بوم ویران ویران شود   چو از شهر ایران نتابی نکوست    که با لشکرش یار باید نخست  

Batch 900 of 5581. Loss:0.29625922441482544. Time:0:06:07
ازان درد بگریست افراسیاب
Example output: ازان درد بگریست افراسیاب  که هنگام آرام یافتی کباب   بباشد همی آفتاب و ماه    مگر کشته شد کشته در زیر گاه  

Batch 1200 of 5581. Loss:0.32953953742980957. Time:0:08:09
بفرمود تا زو بپرسند شاه
Example output: بفرمود تا زو بپرسند شاه  که آمد به ما تاج بر سر کلاه   مگر پیش من سر بپیچد ز داد    نه اندیشد از جان و دل به چیز  

Batch 1500 of 5581. Loss:0.35165882110595703. Time:0:10:12
ترا بهتر آید که فرمان کنی
Example output: ترا بهتر آید که فرمان کنی  زبان را به پیمان

  0%|          | 0/621 [00:00<?, ?it/s]


Validation loss: 0.30382633209228516. Validation Time: 0:01:37

Total training took 1:19:04


In [32]:
save_path = "trained.pth"  
torch.save(model.state_dict(), save_path)
print(f'Model saved at: {save_path}')

Model saved at: trained.pth


## BLEU METRIC TEST 

In [38]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
score = []
model.eval()

for epoch_i in tqdm(range(training_args.num_train_epochs), position=0):
    all_references = []
    all_candidates = []

    for batch in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():
            outputs = model.module.generate(
                input_ids=b_input_ids,
                max_length=50,
                num_return_sequences=5,
                do_sample=True,
                top_k=50,
                top_p=0.95
            )

        references = [tokenizer.decode(ids, skip_special_tokens=True) for ids in b_labels]
        
        for i in range(len(outputs)):
            gen_sample_output = tokenizer.decode(outputs[i], skip_special_tokens=True)
            gen_sample_output = gen_sample_output.replace("<s>", "")
            gen_sample_output = gen_sample_output.replace("</s>", "")
            gen_sample_output = gen_sample_output.replace("<sep>", "\n")
            
            all_references.append([ref.split() for ref in references])
            all_candidates.append(gen_sample_output.split())

    bleu_score = corpus_bleu(all_references, all_candidates)
    score.append(bleu_score)

In [50]:
print(score)

[1.0, 1.0]


# Model Generated Poem Samples

In [51]:

sample_input = df_input[np.random.randint(0, len(df_input))]
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

خود و سرکشان سوی جیحون کشید
Example output: خود و سرکشان سوی جیحون کشید  که از آب دیده رخش ناپدید   یکی بانگ برزد به بیژن چو شیر    که با او به آوردگه گشت دیر  <pad><pad><pad><pad><pad><pad><pad>
Example output: خود و سرکشان سوی جیحون کشید  سراسر ز داد یلان سر کشید   سپهبد چو پردخته شد ز آفرین    سپه را همه خواند و گفت کین  <pad><pad><pad><pad><pad><pad><pad><pad>
Example output: خود و سرکشان سوی جیحون کشید  ز خون مژگان سراسر بر گل کشید   دو زلفش بسان دو نرگس به هم    چو کافور و با نار و سیب و به فندق دژم  
Example output: خود و سرکشان سوی جیحون کشید  ز جیحون گذر کرد و جیحون کشید   چنان شد که پیران بدان جای جنگ    نیاسود و ننمود هرگز نهنگ  <pad><pad><pad><pad><pad><pad><pad>
Example output: خود و سرکشان سوی جیحون کشید  همه کام او شد بی غم و درد و جوش   بدو گفت گر دل نداری فریب    نیابی فریب تو آورد گر نهیب  <pad><pad><pad><pad><pad>


In [69]:
import numpy as np
import torch
from nltk.translate.bleu_score import corpus_bleu

# Assuming you have initialized your model, tokenizer, and have df_input, df_target, and df_concat

index = np.random.randint(0, len(df_input))
sample_input = df_input[index]
print(f'Input sequence: {sample_input}')

sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)
print(f'Labeled target sequence: {df_concat[index]}')

sample_outputs = model.module.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=20
)

print()
print('*' * 50)
print()

# Prepare references for BLEU score calculation
references = [df_concat[index].split()]  # Assuming df_concat[index] is a string

for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output.tolist(), skip_special_tokens=True)
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")
    
    # Calculate BLEU score
    bleu_score = corpus_bleu([references], [gen_sample_output.split()])
    
    print(f'Example output {i + 1}: {gen_sample_output} with BLEU score: {bleu_score}')


Input sequence: بیابان گذارد به اندک سپاه
Labeled target sequence: بیابان گذارد به اندک سپاه <s> شود شاه پیروز و دشمن تباه   بدان ای گزیده شه خسروان    که من هرچ گفتم نباشد جز آن  </s>  

**************************************************

Example output 1: بیابان گذارد به اندک سپاه  اگر کنده یی دور بیند سپاه   ببهرام نامه بد و بی وفا    ابا رای زن پیش جستی رها   with BLEU score: 0.14790264259417688
Example output 2: بیابان گذارد به اندک سپاه  یکی را دهد آب و برگست راه   ز گرد سواران پرخاشجوی    بیازید و ز سرشان بتابید روی   with BLEU score: 0.14765979297062756
Example output 3: بیابان گذارد به اندک سپاه  از ایدر که آیی به شادی به راه   همه لشکر آیدت زار و خوار    همه نامداران که آیدت کار زار   with BLEU score: 0.15337404749451009
Example output 4: بیابان گذارد به اندک سپاه  ز گرد سواران در آن رزمگاه   که او را فگندی به خون ریختن    وزو روی گیتی پر انگیختن   with BLEU score: 0.15371371932217712
Example output 5: بیابان گذارد به اندک سپاه  بدان تا بگیرند بر پشه راه   چو رستم که بودش سپه